In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
# Get miRNA from TargetScan
df  = pd.read_csv('D:/ML4RG_file/genomic_sequence_plus_features_hl_all_tissues.csv', index_col = 0)
rownames = df.index
mirRNA_list = []
for r in rownames[0:10]:
    mirRNA = []
    url = 'http://www.targetscan.org/cgi-bin/targetscan/vert_72/view_gene.cgi?rs=' + r + '&taxid=9606&showcnc=0&shownc=0&shownc_nc=&showncf1=&showncf2=&subset=1'
    if requests.post(url).status_code == 200:
        html = requests.get(url)
        html.encoding = 'utf-8'
        soup = BeautifulSoup(html.text, "html.parser")
        table = soup.find_all('table', id='alt_tstable')
        if len(table)>0:
            line = table[0].find_all('tr')
            for l in line[1:]:
                mirRNA.append(l.text.split()[5][3:])
    print(mirRNA)
    mirRNA_list.append(mirRNA)

df_miRNA = pd.DataFrame()
df_miRNA['id'] = rownames
df_miRNA['miRNA'] = mirRNA_list
df_miRNA.to_csv('C:/Users/yuyu/Desktop/result_miRNA.csv')

In [ ]:
def strTolist(s):
    if s != '[]':
        new_list = []
        element_list = s[1:-1].split(', ')
        for e in element_list:
            new_list.append(e[1:-1])
    else:
        new_list = []
    return new_list

# read a new df
df_RNA = pd.read_csv('C:/Users/yuyu/Desktop/result_miRNA.csv')
miRNA = df_RNA['miRNA']

# get all miRNA's name for each gene in a list 
all_miRNA = []
for mi in miRNA:
    miRNA_list = strTolist(mi)
    if len(miRNA_list)>0:
        for e in miRNA_list:
            if not e.split('.')[0] in all_miRNA:
                all_miRNA.append(e.split('.')[0])
                                     
# crreat a dictionary for {miRNA'name: miRNA family}, get family info from miRBase
miRNA_family_dict = {}

for mir,i in zip(all_miRNA,range(0,len(all_miRNA))):
    family = ''
    url = 'http://www.mirbase.org/cgi-bin/mirna_entry.pl?acc=' + mir
    html = requests.get(url)
    html.encoding = 'utf-8'
    soup = BeautifulSoup(html.text, "html.parser")
    table = soup.find_all('table', class_='data')
    if len(table)>0:
        left = [i.text for i in table[0].find_all('td', class_='left')]
        if 'Gene family' in left:
            family = table[0].find_all('td', class_='right')[left.index('Gene family')].text.split('; ')[1]
        else:
            family =  mir.split('-')[1].lower() + '-' + mir.split('-')[2]
    else:
        mir_1 = mir.split('-')[0] + '-' + mir.split('-')[1] + '-' + mir.split('-')[2]
        url = 'http://www.mirbase.org/cgi-bin/mirna_entry.pl?acc=' + mir_1
        html = requests.get(url)
        html.encoding = 'utf-8'
        soup = BeautifulSoup(html.text, "html.parser")
        table = soup.find_all('table', class_='data')
        if len(table)>0:
            left = [i.text for i in table[0].find_all('td', class_='left')]
            if 'Gene family' in left:
                family = table[0].find_all('td', class_='right')[left.index('Gene family')].text.split('; ')[1]
            else:
                family =  mir.split('-')[1].lower() + '-' + mir.split('-')[2]

    miRNA_family_dict.update({mir:family})

# get unique miRNA family
unique_family = []
for f in miRNA_family_dict:
    if not miRNA_family_dict[f] in unique_family:
        unique_family.append(miRNA_family_dict[f])

# create dataframe of gene and miRNA family to record the number of miRNA family for each gene
df_RNA_family = pd.DataFrame()
df_RNA_family['id'] = df_RNA['id']
for f in unique_family:
    df_RNA_family[f] = [str(i*0) for i in range(len(df_RNA['id']))]
    
miRNA = df_RNA['RNA_family']
transcript = df_RNA['id']
for mi,i in zip(miRNA,range(0,len(transcript))):
    miRNA_list = strTolist(mi)
    if len(miRNA_list)>0:
        for e in miRNA_list:
            this_family = miRNA_family_dict[e.split('.')[0]]
            pre_num = float(df_RNA_family.loc[i,this_family])
            df_RNA_family.loc[i,this_family] = pre_num + 1 
            
df_RNA_family.to_csv('C:/Users/yuyu/Desktop/Number_RNA_family.csv')